# Peroxiredoxin decamer association and dissociation  model

This notebook performs analyses described in the

**Manuscript:**

Modelling the decamerisation cycle of PRDX1 and the inhibition-like effect on its peroxidase activity

**Authors:**

C. Barry, C. Pillay, J. Rohwer

**Purpose:**

Analysis: Simulations with the Prx dimer-decamer model

Plot:  Figure 2

**Requirements:**

Python libraries (see **Imports** below)

Model: Prx dimer-decamer equilibrium model (`Prx_dim-dec.psc`)

##### Imports

In [ ]:
import os

import numpy as np
from matplotlib import pyplot as plt
import matplotlib

import pysces


###### Get directory paths

In [ ]:
prev_dir = os.path.split(os.getcwd())[0]

mod_dir = os.path.join(prev_dir,"models")
if not os.path.isdir(mod_dir): os.mkdir(mod_dir) # ensure dir exists
    
fig_dir = os.path.join(os.getcwd(),"Figures")
if not os.path.isdir(fig_dir): os.mkdir(fig_dir) # ensure dir exists


###### Matplotlib stuff

In [ ]:
%matplotlib inline

multiplier = 1
mpl_width = 4.5*multiplier
mpl_height = 3.5*multiplier
mpl_dpi = 600
mpl_xlabel_fontsize = "large"
mpl_ylabel_fontsize = "large"
mu = "\u03bc"

###### Common variables

In [ ]:
CTC = 0.8 # Critical transition threshold
scan_range = np.linspace(0.1,10,100)

In [ ]:
factors = {"M2":1,
            "M4":2,
            "M6":3,
            "M8":4,
            "M10":5,
          "dimers":1,
          "decamers":5}

## Prx dimer-decamer (figure 2)

In [ ]:
# Load model
mod_Prx_dim_dec = pysces.model("Prx_dim-dec", dir=mod_dir)
mod_Prx_dim_dec.showModel()


In [ ]:
# Adjust koff and kon
Kd = 1.58
kon = 1

mod_Prx_dim_dec.kon = kon
mod_Prx_dim_dec.koff = kon*Kd

The model struggles when Kd and exponent are too high. They both need to be adjusted in order to maintain published steady-state behaviour. This combination reproduces published effects while still being computationally viable.

In [ ]:
# Do sim
def mod_scan(model, scan_range):
    
    model.scan_in = "dimers_init"
    model.scan_out = ['dimers', 'decamers']
    model.Scan1(scan_range)
    
    return model.scan

scan = mod_scan(mod_Prx_dim_dec, scan_range)

In [ ]:
# Plot result
f,axarr = plt.subplots(1,1)
f.set_size_inches(w=mpl_width, h=mpl_height)

ax =  axarr

ax.plot(scan_range,scan.dimers_ss,label = "dimers")
ax.plot(scan_range,scan.decamers_ss,label = "decamers")
ax.plot([CTC,CTC],[0,max(max(scan.dimers_ss),max(scan.decamers_ss))],"k--", label = "CTC")

ax.set_xlabel(f'Total Prx ({mu}M of dimers)',fontsize = mpl_xlabel_fontsize)
ax.set_ylabel(f'Prx oligomer ({mu}M)',fontsize = mpl_ylabel_fontsize)

f.tight_layout()
f.savefig(os.path.join(fig_dir, "SS_dim_dec_species_vs_Prx_init.pdf"),dpi= mpl_dpi)

In [ ]:
# Plot result
f,axarr = plt.subplots(1,1)
f.set_size_inches(w=mpl_width, h=mpl_height)

for species in scan.dtype.names[1:]:
    x_data = scan["dimers_init"]*factors["dimers"]
    x_mol = scan[species]*factors[species.split("_")[0]]/scan["dimers_init"]
    axarr.plot(x_data, x_mol, label = species)

axarr.plot([CTC,CTC],[0,1],"k--", label = "CTC")

axarr.set_xlabel(f'Initial [Peroxiredoxin] ({mu}M)',fontsize = mpl_xlabel_fontsize)
axarr.set_ylabel("f mol",fontsize = mpl_ylabel_fontsize)

f.tight_layout()
